<a href="https://colab.research.google.com/github/JDS289/BaLD4LLM/blob/main/playingAround.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using GPU seems to sometimes cause a CUDA error; using TPU seems to sometimes cause "Unknown crash".
CPU takes approximately 10x longer than GPU.


3B has said False to "Paris is the capital of France". Might want to use 8B instead. For now, I'll continue to use 3B for speed/simplicity. This is also justified for "reproducibility" reasons, though if accuracy is negligible then switch to a better model.


In [2]:
from google.colab import userdata
hf_token = userdata.get("huggingface_secret")

from huggingface_hub import login
login(token=hf_token)

In [3]:
DEFAULT_MODEL = "meta-llama/Llama-3.2-3B-Instruct"

from typing import Optional
import os
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from tqdm.notebook import tqdm

#import warnings
#warnings.filterwarnings('ignore')


# Display first 500 characters of extracted text as preview


device = "cuda" if torch.cuda.is_available() else "cpu"

#import os
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


SYS_PROMPT = ""


#Let's load in the model and start processing the text chunks

accelerator = Accelerator()

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map=device,
    output_hidden_states=True,
    return_dict_in_generate=True,
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)
model.generation_config.pad_token_id = tokenizer.eos_token_id


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
model.generation_config.output_hidden_states = True
model.generation_config.return_dict_in_generate = True
print(model.generation_config)

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "output_hidden_states": true,
  "pad_token_id": 128009,
  "return_dict_in_generate": true,
  "temperature": 0.6,
  "top_p": 0.9
}



In [6]:
def process_chunk(text_chunk):
    """Process a chunk of text and return both input and output for verification"""
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text_chunk},
    ]

    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            temperature=0.7,
            top_p=0.9,
            max_new_tokens=512
        )

    processed_text = tokenizer.decode(output.sequences[0], skip_special_tokens=True).strip()
    processed_text = processed_text[processed_text.index("assistant") + 11:]


    """
    for i in range(len(output.hidden_states)):
        total_shape = np.array(output.hidden_states[i][0].shape)
        for j in range(1, len(output.hidden_states[i])):
            total_shape += np.array(output.hidden_states[i][j].shape)
        print(total_shape)"""
    return processed_text, output.hidden_states

In [9]:
import requests

capitals = requests.get("https://raw.githubusercontent.com/JDS289/BaLD4LLM/refs/heads/main/country-list.csv")
capitals_dict = {}
for string in capitals.content.decode('utf-8').split('\n')[1 : -1]:
  _, country, _, city, _ = string.split('"')
  capitals_dict[country] = city

print(capitals_dict)


{'Afghanistan': 'Kabul', 'Albania': 'Tirana', 'Algeria': 'Algiers', 'Andorra': 'Andorra la Vella', 'Angola': 'Luanda', 'Antigua and Barbuda': "St. John's", 'Argentina': 'Buenos Aires', 'Armenia': 'Yerevan', 'Australia': 'Canberra', 'Austria': 'Vienna', 'Azerbaijan': 'Baku', 'Bahamas': 'Nassau', 'Bahrain': 'Manama', 'Bangladesh': 'Dhaka', 'Barbados': 'Bridgetown', 'Belarus': 'Minsk', 'Belgium': 'Brussels', 'Belize': 'Belmopan', 'Benin': 'Porto-Novo', 'Bhutan': 'Thimphu', 'Bosnia and Herzegovina': 'Sarajevo', 'Botswana': 'Gaborone', 'Brazil': 'Brasília', 'Brunei': 'Bandar Seri Begawan', 'Bulgaria': 'Sofia', 'Burkina Faso': 'Ouagadougou', 'Burundi': 'Bujumbura', 'Cambodia': 'Phnom Penh', 'Cameroon': 'Yaoundé', 'Canada': 'Ottawa', 'Cape Verde': 'Praia', 'Central African Republic': 'Bangui', 'Chad': "N'Djamena", 'Chile': 'Santiago', 'China': 'Beijing', 'Colombia': 'Bogotá', 'Comoros': 'Moroni', 'Costa Rica': 'San José', 'Croatia': 'Zagreb', 'Cuba': 'Havana', 'Cyprus': 'Nicosia', "Côte d'Ivo

In [11]:
statement_list = [f"{capital} is the capital city of {country}" for country, capital in capitals_dict.items()]

def print_ret(x):
  print(x[:2])
  return x

results = [print_ret((statement,) + process_chunk(f'"{statement}". Is this true or false? (Answer ONLY with "True." or "False.")')) for statement in tqdm(statement_list)]

statements = []
responses = []
hidden_statess = []

for s, r, h in results:
  statements.append(s)
  responses.append(r)
  hidden_statess.append(h)

# print(*zip(statements, responses), sep="\n")

  0%|          | 0/187 [00:00<?, ?it/s]

('Kabul is the capital city of Afghanistan', 'False.')
('Tirana is the capital city of Albania', 'True.')
('Algiers is the capital city of Algeria', 'False.')
('Andorra la Vella is the capital city of Andorra', 'False.')
('Luanda is the capital city of Angola', 'False.')
("St. John's is the capital city of Antigua and Barbuda", 'False.')
('Buenos Aires is the capital city of Argentina', 'False.')
('Yerevan is the capital city of Armenia', 'False.')
('Canberra is the capital city of Australia', 'True.')
('Vienna is the capital city of Austria', 'False.')
('Baku is the capital city of Azerbaijan', 'False.')
('Nassau is the capital city of Bahamas', 'True.')
('Manama is the capital city of Bahrain', 'False.')
('Dhaka is the capital city of Bangladesh', 'True.')
('Bridgetown is the capital city of Barbados', 'False.')
('Minsk is the capital city of Belarus', 'True.')
('Brussels is the capital city of Belgium', 'False.')
('Belmopan is the capital city of Belize', 'False.')
('Porto-Novo is t

In [12]:
print(statements)

['Kabul is the capital city of Afghanistan', 'Tirana is the capital city of Albania', 'Algiers is the capital city of Algeria', 'Andorra la Vella is the capital city of Andorra', 'Luanda is the capital city of Angola', "St. John's is the capital city of Antigua and Barbuda", 'Buenos Aires is the capital city of Argentina', 'Yerevan is the capital city of Armenia', 'Canberra is the capital city of Australia', 'Vienna is the capital city of Austria', 'Baku is the capital city of Azerbaijan', 'Nassau is the capital city of Bahamas', 'Manama is the capital city of Bahrain', 'Dhaka is the capital city of Bangladesh', 'Bridgetown is the capital city of Barbados', 'Minsk is the capital city of Belarus', 'Brussels is the capital city of Belgium', 'Belmopan is the capital city of Belize', 'Porto-Novo is the capital city of Benin', 'Thimphu is the capital city of Bhutan', 'Sarajevo is the capital city of Bosnia and Herzegovina', 'Gaborone is the capital city of Botswana', 'Brasília is the capita

In [32]:
# print(len(hidden_statess[0]))
print(hidden_statess[0][0])



# 7 * 29 * 60 * 3072  for EACH hidden_statess[i]

# 7 layers * 29 batches * 60 tokens per input * 3072 activations

# idk why there are 29 batches

(tensor([[[-1.1587e-04,  3.8528e-04, -1.9379e-03,  ...,  2.3937e-04,
          -5.4550e-04,  8.8215e-05],
         [-1.1587e-04,  3.8528e-04, -1.9379e-03,  ...,  2.3937e-04,
          -5.4550e-04,  8.8215e-05],
         [-1.4160e-02,  1.2390e-02,  8.9264e-04,  ..., -1.6785e-03,
           6.3171e-03, -3.3722e-03],
         ...,
         [ 3.8147e-03,  1.7700e-02, -5.1880e-03,  ..., -4.1504e-02,
           8.7891e-03,  8.1787e-03],
         [ 7.5150e-04, -3.2959e-02, -7.7515e-03,  ..., -1.4465e-02,
           1.7578e-02,  1.1292e-02],
         [-1.5137e-02,  1.0071e-02, -4.9438e-03,  ...,  3.7689e-03,
          -1.1475e-02, -2.3346e-03]]], device='cuda:0', dtype=torch.bfloat16), tensor([[[-0.0031,  0.0019,  0.0601,  ...,  0.0510,  0.0021, -0.0049],
         [-0.0031,  0.0019,  0.0601,  ...,  0.0510,  0.0021, -0.0049],
         [-0.0303,  0.0046, -0.0664,  ...,  0.0121, -0.0248,  0.0044],
         ...,
         [-0.0131,  0.0146,  0.0588,  ..., -0.0220, -0.0024, -0.0017],
         [ 0.00

In [22]:
conversation = [
      {"role": "system", "content": SYS_PROMPT},
      {"role": "user", "content": f'"{statements[0]}". Is this true or false? (Answer ONLY with "True." or "False.")'},
]

prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)

In [28]:
print(inputs["input_ids"].shape)

torch.Size([1, 60])
